# Collecting data with SQL

In [1]:
# SQL Alchemy - cria database de todos os bancos de dados

# psycopg2 - apenas de Postgres

import psycopg2 as pg2
import pandas as pd

#nunca coloque suas credenciais no notebook: usar ginger?
#credentials
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
user = 'member'
pwd = 'cdspa'

In [2]:
#conexão com o banco de dados
conn = pg2.connect( 
                    user = user,
                    password = pwd,
                    host = host,
                    port = port,
                    database = database
            )

## 1.0 - Schema Query

In [14]:
#psycopg precisa de um cursor e .execute, ele não retorna dados
cursor = conn.cursor()

query_schema =  """
                SELECT nspname
                FROM pg_catalog.pg_namespace 
                
               """
cursor.execute( query_schema )

#para trazer os dados precisamos do fecth all
record = cursor.fetchall()
#sempre fechar a conexão do banco de dados pois há limite de portas
cursor.close()
conn.close()

## 2.0 - Table Query

In [12]:
cursor = conn.cursor()

query_tables =  """
                SELECT tablename
                FROM pg_tables
                WHERE schemaname = 'pa004'
                
               """
cursor.execute( query_tables )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()

[('users',), ('vehicle',), ('insurance',)]


## 3.0 - Collect data using Psycopg2

In [19]:
cursor = conn.cursor()

query_table_users =  """
                SELECT *
                FROM pa004.users u
                WHERE u.age > 44 
                limit 10
                
               """
cursor.execute( query_table_users )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()

InterfaceError: connection already closed

In [20]:
pd.DataFrame( record )

,0,1,2,3,4
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0
5,15,Male,71,28.0,30.0
6,20,Female,60,33.0,124.0
7,21,Male,65,28.0,124.0
8,22,Male,49,28.0,124.0
9,27,Female,51,28.0,124.0


## 4.0 - Collect data using Pandas

In [8]:
import pandas.io.sql as psql

query_table_users =  """
                SELECT *
                FROM pa004.users u LEFT JOIN pa004.vehicle v ON ( u.id = v.id )
                                   LEFT JOIN pa004.insurance i ON ( u.id = i.id )
                                
               """
df = psql.read_sql( query_table_users, conn )

/tmp/ipykernel_38975/667893799.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql( query_table_users, conn )


In [12]:
#este é o método mais usado
df1 = pd.read_sql( query_table_users, conn )
df1 = df1.drop( df1.columns[[5,9]], axis=1 )
df1.shape

/tmp/ipykernel_38975/4045162626.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql( query_table_users, conn )


(381109, 11)

In [13]:
df1.head()

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Male,23,11.0,152.0,1,< 1 Year,Yes,0,23367.0,249,0
1,Female,41,15.0,14.0,1,1-2 Year,No,1,31409.0,221,0
2,Female,25,35.0,152.0,1,< 1 Year,No,1,46622.0,299,0
3,Female,26,8.0,160.0,1,< 1 Year,No,0,2630.0,136,0
4,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0
